# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_TEST_KEY_KdR')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HF_TOKEN')



In [5]:
#!pip install pandas

In [3]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [4]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
!pip install langchain_community

In [5]:


import langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [6]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature= 1.1, model_name = 'gpt-3.5-turbo', openai_api_key=OPENAI_API_KEY) # Temperature is set to 1.1 to allow for more creative responses


In [7]:
prompt = ChatPromptTemplate.from_template( #Write a query that would take a variable to describe any product
 " You are a sarcastic and humorous product description writer. You will be given a product name, and you will generate a 5 line product description that is both funny and sarcastic, based on the user reviews of the product. \n\n"
 "Product name: {product_name}\n\n"
)

In [8]:

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\karel\AppData\Local\Temp\ipykernel_15996\546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [9]:
# Generate a list of product names from the DataFrame
product_names = df['Product'].tolist()
import random

# Randomly select a product name from the DataFrame
random_index = random.randint(0, len(product_names) - 1)

product =  product_names[random_index] # Randomly select a product name from the DataFrame
chain.run(product)

C:\Users\karel\AppData\Local\Temp\ipykernel_15996\512106262.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'"Are you tired of waiting for water to boil the old-fashioned way? Look no further than the Hervidor de Agua Eléctrico! It may not make your morning coffee any faster, but hey, at least it looks sleek on your countertop. Customer reviews rave about its ability to do the one job it was designed for - boiling water. So if you love paying for hot water and a fancy appliance, this is the product for you! Just don\'t expect it to do your taxes too."'

## SimpleSequentialChain

In [10]:
from langchain.chains import SimpleSequentialChain

In [21]:


# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
     " You are a glowing product description writer. You will be given a product name, and you will generate a 5 line product description that is over the top positive, based on the user reviews of the product. \n\n"
 "Product name: {product_name}\n\n"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [22]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    " You are a sarcastic and humorous product salesman. You will be given a product review (review #1), and you will generate a 5 line recommendation (that is both funny and sarcastic) for a product that the user might " \
    "also be interested in (review #2)."
    "Product review #1: {input}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [23]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [24]:
result = overall_simple_chain.run(product)
print(result)



> Entering new SimpleSequentialChain chain...
Introducing the Hervidor de Agua Eléctrico, the must-have kitchen appliance that will revolutionize your mornings! Customers rave about its lightning-fast boiling speed, sleek design, and effortless pour spout for mess-free serving. Get ready to enjoy perfectly heated water for your favorite beverages in seconds, thanks to the convenient automatic shut-off feature. Experience the ultimate convenience and efficiency with the Hervidor de Agua Eléctrico!
Recommendation #2: Do you love fast-boiling water but wish you could spice up your mornings even more? Say goodbye to boring beverages and hello to the Flamethrower Coffee Maker! Watch in awe as this powerful machine shoots flames into your cup, turning your morning routine into an adrenaline-pumping experience. With its sleek and dangerously edgy design, this coffee maker will surely awaken your senses and ignite your day! Who needs a ordinary kettle when you can have a flamethrower?

> Fin

**Repeat the above twice for different products**

## SequentialChain

In [25]:
from langchain.chains import SequentialChain

In [26]:
llm = ChatOpenAI(temperature=0.9, model_name='gpt-3.5-turbo', api_key=OPENAI_API_KEY)


first_prompt = ChatPromptTemplate.from_template(
  "Act as a translator that translates a product review from any given lanugage to English. \n\n"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key= "translation" #Give a name to your output
                    )


In [29]:
second_prompt = ChatPromptTemplate.from_template(
    "Act as a product review writer. You will be given a product review in English, please generate a 3 line product review that is witty and engaging based on the review input. \n\n"
    "Translation: {translation}\n\n"  # Use the output key from the first chain
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="review"
                    )


In [30]:
# prompt template 3: translate to english or other language (already done in chain_one, instead decided to create a critic bot)
third_prompt = ChatPromptTemplate.from_template(
    " Act as a review critic. You will be given a product review, and generate a 3 line critique of the review, including the language used and the overall tone. \n\n"
    "Product review: {review}\n\n"  # Use the output key from the second chain
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="critique"
                      )


In [32]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "Act as a product review salesman. You will be given a product review and a critique of the review. Based on the review and the critique, generate an enthusiastic but honest sales pitch for the product. \n\n"
        "Product review: {review}\n\n" # Use the output key from the second chain
        "Critique: {critique}\n\n"  # Use the output key from the third chain
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="sales_pitch"
                     )


In [36]:
# overall_chain: input= Review 
# and output = English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["input"],  # Input variable for the first chain 
      # Output variables for the overall chain
    output_variables=["translation", "review", "critique", "sales_pitch"],
    verbose=True
)

In [37]:
review = df.Review[5]
overall_chain(review)

C:\Users\karel\AppData\Local\Temp\ipykernel_15996\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'input': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'translation': 'Original review (French): \n"Ce produit est incroyable! Il est très facile à utiliser et les résultats sont vraiment époustouflants. Je le recommande à tous ceux qui recherchent une solution efficace pour leur peau."\n\nTranslated review (English): \n"This product is amazing! It is very easy to use and the results are truly stunning. I recommend it to anyone looking for an effective solution for their skin."',
 'review': '"Experience the incredible results with this easy-to-use skincare product. Say goodbye to dull skin and hello to a radiant glow. A must-have for anyone in search of a skincare game-changer!"',
 'critique': 'Critique: The review uses hyperbolic language in describing the product as "incredible" and a "game-changer," which could come across as exaggerated and insincere. The ove

**Repeat the above twice for different products or reviews**

## Router Chain

In [38]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [39]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [40]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [41]:
llm = ChatOpenAI(temperature= 0.1, model_name = 'gpt-3.5-turbo', openai_api_key=OPENAI_API_KEY)

In [42]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [43]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [44]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [45]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [46]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\karel\AppData\Local\Temp\ipykernel_15996\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [48]:
chain.run("What is black body radiation?")
chain.run("What is a black body in thermal physics?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


> Entering new MultiPromptChain chain...
physics: {'input': 'What is a black body in thermal physics?'}
> Finished chain.


'A black body in thermal physics is an idealized physical body that absorbs all incident electromagnetic radiation, regardless of frequency or angle of incidence. It also emits radiation at all frequencies and temperatures. This concept is used as a theoretical model to study the behavior of real objects in thermal equilibrium.'

In [49]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [50]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell growth, and repair. Having DNA in every cell allows for the coordination of these processes and ensures that the organism functions properly as a whole. \n\nIn summary, every cell in our body contains DNA because it is essential for the proper functioning and development of all living organisms.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**